In [ ]:
import cv2
import os
import time
import imutils

# ============================================================
#      				  Hyper-parameters
# ============================================================

CONTRAST = False
DEBUG = False
DENOISING = False
BOUNDING_BOX = True
CONTOUR_AREA = 1000

xz=50
yz=xz
folder_path = './data/'       									
cropped_path = './data_seg/seg3 -30APR -20 -1500 -WC/'

# ============================================================

In [ ]:
def extract(image,path,frame_count,img_name,category,xz=0,yz=0):

    print("\n==================== Image {}=================\n".format(frame_count))
    img = cv2.imread(image)
    img_float = np.float32(img)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10 , 1)
    
    k=50
    ret, label, centers = cv2.kmeans(img_float, k, None, criteria, 50, cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(centers)
    res = center[label.flatten()]                         
    res2 = res.reshape(img.shape)
    if DEBUG: cv2.imshow("K Means", res2)
        
    meanshift = cv2.pyrMeanShiftFiltering(img, sp=8, sr=16, maxLevel=1, termcrit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 5, 1))
    if DEBUG: cv2.imshow("Output of meanshift", meanshift)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    fg = cv2.erode(thresh, None, iterations=1)
    bgt = cv2.dilate(thresh, None, iterations=1)
    ret, bg = cv2.threshold(bgt, 1, 128, 1)
    
    marker = cv2.add(fg, bg)
    canny = cv2.Canny(marker, 110, 150)
    new, contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    marker32 = np.int32(marker)
    cv2.watershed(img,marker32)
    m = cv2.convertScaleAbs(marker32)
    ret, thresh = cv2.threshold(m, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    thresh_inv = cv2.bitwise_not(thresh)
    res = cv2.bitwise_and(img, img, mask=thresh)
    res3 = cv2.bitwise_and(img, img, mask=thresh_inv)
    res4 = cv2.addWeighted(res, 1, res3, 1, 0)
    final = cv2.drawContours(res4, contours, -1, (0, 255, 0), 1)

    cv2.imshow("Watershed", final)
    
    # =================================================
    image, contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if not os.path.exists(path):
        os.makedirs(path)
    
    o_img= img.copy()
    count=1
    
    for c in contours:
        
        crop_path = path
        if cv2.contourArea(c) >= CONTOUR_AREA:
            x, y, w, h = cv2.boundingRect(c)
            roi = img[y-yz:y + h+ yz, x-xz:x + w+ xz]
            if BOUNDING_BOX: cv2.rectangle(o_img,(x,y),(x+w,y+h),(0,255,0),2)     

            if 0 not in roi.shape:
                print("[INFO]: Image {}: Shape: {} Area: ({}) Location: ({},{},{},{})...".format(count, roi.shape, cv2.contourArea(c),x,y,w,h))
                crop_path+='/{}_{}.{}.jpg'.format(category,frame_count,count)
                cv2.imwrite(crop_path,roi)# % count,roi)
                count+=1

    if BOUNDING_BOX:

        crop_path+='/{}_{}.jpg'.format(category,frame_count)
        o_img = imutils.resize(o_img, width=500)
        cv2.imwrite(crop_path,o_img)


# ============================================================

start = time.time()
list_dir = os.listdir(folder_path)
print("Grains categories:",list_dir,'\n')
f_cnt=0


for category in sorted(list_dir):
    
    crop_path=cropped_path
    file = folder_path + category + '/'
    crop_path = crop_path + category + '/'

    for image in sorted(os.listdir(file)):

        img_name = image
        image = os.path.join(file,image)
        path = crop_path + img_name.split('.')[0]
        crop_path = crop_path
        f_cnt+=1
        extract(image,path,f_cnt,img_name,category,xz,yz)
    
end = time.time()
print("Extraction Time:",end-start,'s')
print("Cropping Done!")

# ==================